In [35]:
import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

#username = os.environ['RENKU_USERNAME']
username = "eric"
hiveaddr = os.environ['HIVE_SERVER2']
(hivehost,hiveport) = hiveaddr.split(':')
print("Operating as: {0}".format(username))

Operating as: eric


In [36]:
from pyhive import hive

# Create the connection
conn = hive.connect(host=hivehost, 
                    port=hiveport,
                    username=username) 
# Create the cursor
cur = conn.cursor()

In [37]:
### Create your database if it does not exist
query = """
CREATE DATABASE IF NOT EXISTS {0} LOCATION '/group/five-guys/hive'
""".format(username)
cur.execute(query)

In [12]:
def retrieve_df_connections(table_name):
    ### Create your database if it does not exist
    
    query = """
    DROP TABLE IF EXISTS {0}
    """.format(table_name)
    cur.execute(query)
    
    ### Creation of the table
    query = """
    CREATE EXTERNAL TABLE {0}(
        departure_id string,
        arrival_id string,
        departure_time string,
        arrival_time string,
        trip_id string,
        monday string,
        tuesday string,
        wednesday string,
        thursday string,
        friday string
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    location '/group/five-guys/{0}'
    tblproperties ("skip.header.line.count"="1")
    """.format(table_name)
    cur.execute(query)
    
    query = """
    SELECT *
    FROM {0}.csv
    """.format(table_name)
    df = pd.read_sql(query, conn)
    df.columns = [column.split('.')[1] for column in df.columns]
    
    df["departure_time"] = df["departure_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

    df["arrival_time"] = df["arrival_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))
    
    df[['monday', 'tuesday', 'wednesday', 'thursday', 'friday']] = df[['monday', 'tuesday', 'wednesday', 'thursday', 'friday']].replace({'0':False, '1':True})

    result = list(df.itertuples(index=False, name=None))
    
    del df
    
    return result



In [13]:
conn_table = retrieve_df_connections("conn_table")

OperationalError: TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=['*org.apache.hive.service.cli.HiveSQLException:Error while processing statement: FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. MetaException(message:java.security.AccessControlException: Permission denied: user=ceraolo, access=WRITE, inode="/group/five-guys/conn_table":eric:hadoop:drwxr-xr-x\n\tat org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:399)\n\tat org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:261)\n\tat org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:193)\n\tat org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1857)\n\tat org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1841)\n\tat org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPathAcces

In [ ]:
import pickle
path_data = "../data/"
pickle.dump(conn_table, open(path_data+"connections_data.pickle", "wb"))

In [38]:
def retrieve_df_trips(table_name):
    ### Create your database if it does not exist
    
    query = """
    DROP TABLE IF EXISTS {0}
    """.format(table_name)
    cur.execute(query)
    
    ### Creation of the table
    query = """
    CREATE EXTERNAL TABLE {0}(
        route_id string,
        service_id string,
        trip_id string,
        trip_headsign string,
        trip_short_name string,
        direction_id string
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    location '/group/five-guys/{0}'
    tblproperties ("skip.header.line.count"="1")
    """.format(table_name)
    cur.execute(query)
    
    query = """
    SELECT *
    FROM {0}
    """.format(table_name)
    df = pd.read_sql(query, conn)
    df.columns = [column.split('.')[1] for column in df.columns]

    routes_tot_filtered_map = df.groupby('trip_id')[['route_id', 'service_id', 'trip_headsign', 'trip_short_name', 'direction_id']].apply(lambda g: g.values.tolist()).to_dict()
    names_cols = ["route_id", "service_id", "trip_headsign", "trip_short_name", "direction_id"]

    map_copied = routes_tot_filtered_map.copy()

    for k,v in routes_tot_filtered_map.items():
        newdict = {}
        for ind, element in enumerate(v[0]):
            newdict[names_cols[ind]] = element
        map_copied[k] = newdict
    
    del df
    
    return map_copied




In [39]:
trips = retrieve_df_trips("trips")

In [41]:
import pickle
path_data = "../data/"
pickle.dump(trips, open(path_data+"trips.pickle", "wb"))

In [115]:
def retrieve_df_stops(table_name):
    ### Create your database if it does not exist
    
    query = """
    DROP TABLE IF EXISTS {0}.csv
    """.format(table_name)
    cur.execute(query)
    
    ### Creation of the table
    query = """
    CREATE EXTERNAL TABLE {0}(
        stop_id string,
        stop_name string,
        stop_lat string,
        stop_lon string,
        parent_station string
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ';'
    STORED AS TEXTFILE
    location '/group/five-guys/{0}'
    tblproperties ("skip.header.line.count"="1")
    """.format(table_name)
    cur.execute(query)
    
    query = """
    SELECT *
    FROM {0}
    """.format(table_name)
    df = pd.read_sql(query, conn)
    df.columns = [column.split('.')[1] for column in df.columns]
    
    return df


In [116]:
stops = retrieve_df_stops("stops_table")

,stop_id,stop_name,stop_lat,stop_lon,parent_station
0,1100150,"Schlächtenhaus, Schule",47.6898550981112,7.74233688977719,
1,1100376,"Brombach (LÖ), Friedhof",47.6314693778999,7.68696473565635,
2,1100411,"Häg-Ehrsberg, Ehrsberg",47.74319213737,7.90619858077624,
3,1101439,"Gailingen, Schule",47.6948133619046,8.75378600091951,
4,1101511,"Hoppetenzell, Sartorystrasse",47.8931911144564,9.01140485813517,


In [118]:
def process_stops(df):
    close_stops_map = df.groupby('stop_id')[['stop_name', 'stop_lat', 'stop_lon', 'parent_station']].apply(lambda g: g.values.tolist()).to_dict()
    names_cols = ['stop_name', 'stop_lat', 'stop_lon', 'parent_station']

    stops_final = close_stops_map.copy()

    for k,v in close_stops_map.items():
        newdict = {}
        for ind, element in enumerate(v[0]):
            newdict[names_cols[ind]] = element
            if names_cols[ind] == "parent_station" and isinstance(element, str) and element!="":
                if element[:6]=="Parent":
                    element = element[6:]
                    newdict[names_cols[ind]] = element
                if element[-1]=="P":
                    element = element[:-1]
                    newdict[names_cols[ind]] = element
                if element != str(int(element)):
                    print(element)
        stops_final[k] = newdict
    
    return stops_final

In [119]:
stops_dict = process_stops(stops)

{'1100006': {'stop_name': 'Zell (Wiesental), Bahnhof',
  'stop_lat': '47.7046317515335',
  'stop_lon': '7.84777215468897',
  'parent_station': ''},
 '1100008': {'stop_name': 'Zell (Wiesental), Wilder Mann',
  'stop_lat': '47.7100842702352',
  'stop_lon': '7.85964788274668',
  'parent_station': ''},
 '1100009': {'stop_name': 'Zell (Wiesental), Grönland',
  'stop_lat': '47.7131911044794',
  'stop_lon': '7.86290876722849',
  'parent_station': ''},
 '1100010': {'stop_name': 'Atzenbach',
  'stop_lat': '47.7146175266411',
  'stop_lon': '7.8723500608659',
  'parent_station': ''},
 '1100011': {'stop_name': 'Mambach, Brücke',
  'stop_lat': '47.7282088873189',
  'stop_lon': '7.8774704579861',
  'parent_station': ''},
 '1100012': {'stop_name': 'Mambach, Mühlschau',
  'stop_lat': '47.7340818684375',
  'stop_lon': '7.8813871126254',
  'parent_station': ''},
 '1100013': {'stop_name': 'Mambach, Silbersau',
  'stop_lat': '47.7395192233867',
  'stop_lon': '7.88223152899259',
  'parent_station': ''},
 '

In [120]:
import pickle
path_data = "../data/"
pickle.dump(stops_dict, open(path_data+"stops.pickle", "wb"))

In [45]:
def retrieve_df_footpath(table_name):
    ### Create your database if it does not exist
    
    query = """
    DROP TABLE IF EXISTS {0}
    """.format(table_name)
    cur.execute(query)
    
    ### Creation of the table
    query = """
    CREATE EXTERNAL TABLE {0}(
        departure_id string,
        arrival_id string,
        `time` int
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ';'
    STORED AS TEXTFILE
    location '/group/five-guys/{0}.csv'
    tblproperties ("skip.header.line.count"="1")
    """.format(table_name)
    cur.execute(query)
    
    query = """
    SELECT *
    FROM {0}
    """.format(table_name)
    df = pd.read_sql(query, conn)
    df.columns = [column.split('.')[1] for column in df.columns]
    
    footpath_map = df.groupby('arrival_id')[['departure_id', 'time']].apply(lambda g: g.values.tolist()).to_dict()
    footpath_map = {k: dict(v) for k, v in footpath_map.items()}
    
    
    del df
    
    return footpath_map






In [ ]:
footpath = retrieve_df_footpath("footpath")

In [ ]:
pickle.dump(footpath, open(path_data+"footpath.pickle", "wb"))

In [25]:

#TO GET OBJECTS FROM PICKLE
file = open(path_data+"trips.pickle",'rb')
object_file = pickle.load(file)
file.close()
#object_file

In [26]:
object_file

{'TA+b0001': {'route_id': '9.TA.1-217-j19-1.1.H',
  'service_id': '1-217-j19-1',
  'trip_headsign': '"Affoltern a. A.',
  'trip_short_name': ' Bahnhof"',
  'direction_id': '21719'},
 'TA+b0003': {'route_id': '119.TA.1-305-j19-1.1.H',
  'service_id': '1-305-j19-1',
  'trip_headsign': 'Kindhausen AG',
  'trip_short_name': '4826',
  'direction_id': '0'},
 'TA+b0008': {'route_id': '10.TA.1-10-j19-1.1.R',
  'service_id': '1-10-j19-1',
  'trip_headsign': '"Killwangen',
  'trip_short_name': ' Bahnhof"',
  'direction_id': '3819'},
 'TA+b000k': {'route_id': '513.TA.26-10-j19-1.6.R',
  'service_id': '26-10-j19-1',
  'trip_headsign': '"Zürich Flughafen',
  'trip_short_name': ' Fracht"',
  'direction_id': '2978'},
 'TA+b000n': {'route_id': '3.TA.18-46-j19-1.3.H',
  'service_id': '18-46-j19-1',
  'trip_headsign': 'Erstfeld',
  'trip_short_name': '2421',
  'direction_id': '0'},
 'TA+b0012': {'route_id': '12.TA.1-231-j19-1.1.H',
  'service_id': '1-231-j19-1',
  'trip_headsign': '"Jonen',
  'trip_shor